In [1]:
import pandas as pd
import urllib.request, json
import plotly_express as px
import plotly.figure_factory as ff

In [2]:
with urllib.request.urlopen('https://api.mfapi.in/mf') as url:
    data = json.load(url)
df_mfs = pd.DataFrame(data)

In [3]:
df_mfs[df_mfs.schemeName.str.contains('HDFC Flex')]

,schemeCode,schemeName
1202,101762,HDFC Flexi Cap Fund - Growth Option
1204,101762,HDFC Flexi Cap Fund - Growth Plan
1205,101763,HDFC Flexi Cap Fund - IDCW Plan
1206,101763,HDFC Flexi Cap Fund - Dividend Option
16159,118954,HDFC Flexi Cap Fund -Direct Plan - Dividend Op...
16160,118954,HDFC Flexi Cap Fund - IDCW Option - Direct Plan
16162,118955,HDFC Flexi Cap Fund -Direct Plan - Growth Option
16163,118955,HDFC Flexi Cap Fund - Growth Option - Direct Plan


In [4]:
df_mfs[df_mfs.schemeName.str.startswith('Parag')]

,schemeCode,schemeName
20226,122639,Parag Parikh Long Term Equity Fund - Direct Pl...
20227,122639,Parag Parikh Flexi Cap Fund - Direct Plan - Gr...
20228,122640,Parag Parikh Long Term Equity Fund - Regular P...
20229,122640,Parag Parikh Flexi Cap Fund - Regular Plan - G...
35210,143260,Parag Parikh Liquid Fund- Regular Plan- Growth
35211,143261,Parag Parikh Liquid Fund- Regular Plan- Monthl...
35212,143261,Parag Parikh Liquid Fund- Regular Plan- Monthl...
35213,143262,Parag Parikh Liquid Fund- Direct Plan- Monthly...
35214,143262,Parag Parikh Liquid Fund- Direct Plan- Monthly...
35215,143263,Parag Parikh Liquid Fund- Direct Plan- Daily R...


In [5]:
def get_nav(scheme_code = '122639'):
    # scheme_code = '122639'
    mf_url = 'https://api.mfapi.in/mf/' + scheme_code
    with urllib.request.urlopen(mf_url) as url:
        data = json.load(url)

    df_navs = pd.DataFrame(data['data'])
    df_navs['date'] = pd.to_datetime(df_navs.date, format='%d-%m-%Y')
    df_navs['nav'] = df_navs['nav'].astype(float)
    df_navs = df_navs.sort_values(['date']).set_index(['date'])
    df_dates = pd.DataFrame(pd.date_range(start=df_navs.index.min(), end=df_navs.index.max()), columns=['date']).set_index(['date'])
    df_navs = df_navs.join(df_dates, how='outer').ffill().reset_index()
    return df_navs

In [6]:
df_navs = get_nav()

In [7]:
base_date = '2015-01-01'
base_nav = df_navs.set_index(['date']).loc[base_date:].nav[0]
df_sel_navs = df_navs[df_navs['date'] >= base_date].copy()
df_sel_navs

,date,nav
583,2015-01-01,16.2594
584,2015-01-02,16.3896
585,2015-01-03,16.3896
586,2015-01-04,16.3896
587,2015-01-05,16.3987
...,...,...
3433,2022-10-21,51.5748
3434,2022-10-22,51.5748
3435,2022-10-23,51.5748
3436,2022-10-24,51.5748


In [8]:
df_navs[df_navs['date']]

KeyError: "None of [DatetimeIndex(['2013-05-28', '2013-05-29', '2013-05-30', '2013-05-31',\n               '2013-06-01', '2013-06-02', '2013-06-03', '2013-06-04',\n               '2013-06-05', '2013-06-06',\n               ...\n               '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19',\n               '2022-10-20', '2022-10-21', '2022-10-22', '2022-10-23',\n               '2022-10-24', '2022-10-25'],\n              dtype='datetime64[ns]', length=3438, freq=None)] are in the [columns]"

In [ ]:
px.line(df_navs, x = 'date', y='nav', log_y=True)

In [ ]:
def get_cagr(df_navs_orig, num_years = 1):
    df_navs = df_navs_orig.copy()
    df_navs['prev_nav'] = df_navs.nav.shift(365 * num_years)
    df_navs = df_navs.dropna()
    df_navs['returns'] = df_navs['nav'] / df_navs['prev_nav'] - 1
    df_navs['cagr'] = 100 * ((1 + df_navs['returns']) ** (1 / num_years) - 1)
    df_navs['years'] = num_years
    df_cagr = df_navs[['date', 'years', 'cagr']]
    return df_cagr

In [ ]:
years = [x + 1 for x in range(9)]
list_cagr = []
for y in years:
    df_cagr = get_cagr(df_navs, y)
    list_cagr.append(df_cagr)
df_cagrs = pd.concat(list_cagr)

In [ ]:
df_cagrs

In [ ]:
px.line(df_cagrs, x='date', y='cagr', color='years')

In [ ]:
df_cagrs.groupby('years').describe()

In [ ]:
px.density_contour(df_cagrs.set_index(['date']), x='cagr', color='years')

In [ ]:
# df_cagrs[df_cagrs['years'] == 9]
px.histogram(df_cagrs, x='cagr', color='years')

In [ ]:
df_cagrs['cagr_bucket'] = round(df_cagrs['cagr'] / 1, 0) * 1

In [ ]:
df_total = pd.DataFrame(df_cagrs.groupby(['years']).cagr.count()).reset_index()
df_total.columns = ['years', 'total']

In [ ]:
df_counts = pd.DataFrame(df_cagrs.groupby(['years', 'cagr_bucket']).cagr.count()).reset_index()
df_counts.columns = ['years', 'cagr_bucket', 'count']

In [ ]:
df_x = df_counts.merge(df_total, on='years', how='left')
df_x['pct'] = df_x['count'] / df_x['total']

In [ ]:
df_x['years'] = df_x['years'].astype(str)
px.bar(df_x, x='cagr_bucket', y='pct', color='years', barmode='overlay')

In [ ]:
df_x

In [ ]:
df_cagrs[df_cagrs['years']==1].cagr

In [ ]:
# Next Plans
# SIP, SWP and STP analysis

In [1]:
import pandas as pd

In [3]:
df_sep = pd.read_excel('https://portal.amfiindia.com/spages/amsep2022repo.xls')
df_sep.columns = df_sep.loc[1]
title_sep = df_sep.loc[0][0]
df_s = df_sep.iloc[2:, :]

In [23]:
df_sep.iloc[:25, :5]

1,Sr,Scheme Name,"No. of Schemes as on September 30, 2022","No. of Folios as on September 30, 2022",Funds Mobilized for the month of September 2022 (INR in crore)
0,Monthly Report for the month of September 2022,NaN,NaN,NaN,NaN
1,Sr,Scheme Name,"No. of Schemes as on September 30, 2022","No. of Folios as on September 30, 2022",Funds Mobilized for the month of September 202...
2,A,Open ended Schemes,NaN,NaN,NaN
3,I,Income/Debt Oriented Schemes,NaN,NaN,NaN
4,i,Overnight Fund,32,611525,552502.991719
5,ii,Liquid Fund,37,1753272,299102.844183
6,iii,Ultra Short Duration Fund,26,621847,13749.007249
7,iv,Low Duration Fund,22,986513,7331.801119
8,v,Money Market Fund,22,420264,21219.657995
9,vi,Short Duration Fund,26,530585,3508.970477


In [37]:
import numpy as np

In [41]:
sr_vals = np.array([x[0] for x in list(df_sep[['Sr ']].values)])

In [104]:
np.where(sr_vals == 'V')

(array([50], dtype=int64),)

In [48]:
val = sr_vals[5]

In [49]:
val

'ii'

In [51]:
np.where(sr_vals == val)

(array([ 5, 24, 38, 47, 52, 62, 69], dtype=int64),)

In [62]:
df_x = df_s[['Sr ', 'Scheme Name ']]

In [132]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [68]:
df_w = df_x.pivot(columns='Sr ', values='Scheme Name ')

In [70]:
df_y = df_w.ffill()

In [150]:
df_y

Sr,NaN,A,B,C,I,II,III,IV,V,i,ii,iii,iv,ix,v,vi,vii,viii,x,xi,xii,xiii,xiv,xv,xvi
2,NaN,Open ended Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,Short Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,Short Duration Fund,Medium Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,Short Duration Fund,Medium Duration Fund,Medium to Long Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df_y[['A', 'B', 'C']].iloc[11,:].dropna().values[-1]

'Open ended Schemes'

In [79]:
df_y[['I', 'II', 'III', 'IV']].iloc[11,:].dropna().values[-1]

'Income/Debt Oriented Schemes'

25

In [111]:
l1 = np.where(df_y.columns == 'A')[0][0]
l2 = np.where(df_y.columns == 'I')[0][0]
l3 = np.where(df_y.columns == 'i')[0][0]
l4 = len(df_y.columns)

In [98]:
l1, l2, l3

(1, 4, 9)

In [99]:
import string

In [114]:
l1_names = list(string.ascii_uppercase[0 : l2-l1])
l2_names = [getRoman(x + 1, 'caps') for x in range(0, l3-l2)]
l3_names = [getRoman(x + 1) for x in range(0, l4-l3)]

In [134]:
l1, l2, l3, l4

(1, 4, 9, 25)

In [139]:
row_num = 88
l1_val = df_y[l1_names].iloc[row_num,:].dropna().values[-1]
l2_val = df_y[l2_names].iloc[row_num,:].dropna().values[-1]
l3_val = df_y[l3_names].iloc[row_num,:].dropna().values[-1]
l1_val, l2_val 

('Interval Schemes', 'Other Schemes')

In [142]:
pd.options.mode.chained_assignment = None  # default='warn'

In [148]:
for row_num in range(2, 89):
    l1_val = df_y[l1_names].iloc[row_num,:].dropna().values[-1]
    l2_val = df_y[l2_names].iloc[row_num,:].dropna().values[-1]
    df_s.loc[row_num, 'Level1'] = l1_val
    df_s.loc[row_num, 'Level2'] = l2_val

In [169]:
df_s

1,Sr,Scheme Name,"No. of Schemes as on September 30, 2022","No. of Folios as on September 30, 2022",Funds Mobilized for the month of September 2022 (INR in crore),Repurchase/ Redemption for the month of September 2022 (INR in crore),Net Inflow (+ve)/Outflow (-ve) for the month of September 2022 (INR in crore),"Net Assets Under Management as on September 30, 2022 (INR in crore)",Average Net Assets Under Management for the month September 2022 (INR in crore),"No. of segregated portfolios created as on September 30, 2022","Net Assets Under Management in segregated portfolio as on September 30, 2022 (INR in crore)",Level1,Level2
2,A,Open ended Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Income/Debt Oriented Schemes
3,I,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Income/Debt Oriented Schemes
4,i,Overnight Fund,32,611525,552502.991719,519374.658639,33128.33308,141319.784048,133564.031977,0,0,Open ended Schemes,Income/Debt Oriented Schemes
5,ii,Liquid Fund,37,1753272,299102.844183,359073.141476,-59970.297293,352213.142253,419855.510365,0,0,Open ended Schemes,Income/Debt Oriented Schemes
6,iii,Ultra Short Duration Fund,26,621847,13749.007249,22202.813311,-8453.806062,89480.736951,96696.217402,0,0,Open ended Schemes,Income/Debt Oriented Schemes
7,iv,Low Duration Fund,22,986513,7331.801119,14991.647854,-7659.846736,95673.47909,101883.125742,1,0,Open ended Schemes,Income/Debt Oriented Schemes
8,v,Money Market Fund,22,420264,21219.657995,32451.719748,-11232.061754,107379.297939,115590.114297,0,0,Open ended Schemes,Income/Debt Oriented Schemes
9,vi,Short Duration Fund,26,530585,3508.970477,4539.825665,-1030.855188,97272.496918,99473.387959,0,0,Open ended Schemes,Income/Debt Oriented Schemes
10,vii,Medium Duration Fund,15,258984,266.209468,935.289637,-669.080169,28203.83209,28727.488527,3,0,Open ended Schemes,Income/Debt Oriented Schemes
11,viii,Medium to Long Duration Fund,12,108680,43.509468,133.088444,-89.578976,8926.795229,9029.591409,0,0,Open ended Schemes,Income/Debt Oriented Schemes


In [170]:
df_s.loc[[x + 2 for x in list(np.where(df_s['No. of Schemes as on September 30, 2022'].isna())[0])], :]

1,Sr,Scheme Name,"No. of Schemes as on September 30, 2022","No. of Folios as on September 30, 2022",Funds Mobilized for the month of September 2022 (INR in crore),Repurchase/ Redemption for the month of September 2022 (INR in crore),Net Inflow (+ve)/Outflow (-ve) for the month of September 2022 (INR in crore),"Net Assets Under Management as on September 30, 2022 (INR in crore)",Average Net Assets Under Management for the month September 2022 (INR in crore),"No. of segregated portfolios created as on September 30, 2022","Net Assets Under Management in segregated portfolio as on September 30, 2022 (INR in crore)",Level1,Level2
2,A,Open ended Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Income/Debt Oriented Schemes
3,I,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Income/Debt Oriented Schemes
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Growth/Equity Oriented Schemes
22,II,Growth/Equity Oriented Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Growth/Equity Oriented Schemes
35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Hybrid Schemes
36,III,Hybrid Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Hybrid Schemes
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Solution Oriented Schemes
45,IV,Solution Oriented Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Solution Oriented Schemes
49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Other Schemes
50,V,Other Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Open ended Schemes,Other Schemes


In [133]:
df_y

Sr,NaN,A,B,C,I,II,III,IV,V,i,ii,iii,iv,ix,v,vi,vii,viii,x,xi,xii,xiii,xiv,xv,xvi
2,NaN,Open ended Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,Short Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,Short Duration Fund,Medium Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,Open ended Schemes,NaN,NaN,Income/Debt Oriented Schemes,NaN,NaN,NaN,NaN,Overnight Fund,Liquid Fund,Ultra Short Duration Fund,Low Duration Fund,NaN,Money Market Fund,Short Duration Fund,Medium Duration Fund,Medium to Long Duration Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# Function to convert integer to Roman values
def getRoman(number, mode='small'):
    num = [1, 4, 5, 9, 10, 40, 50, 90,
        100, 400, 500, 900, 1000]
    sym = ["I", "IV", "V", "IX", "X", "XL",
        "L", "XC", "C", "CD", "D", "CM", "M"]
    if mode != 'caps':
        sym = [x.lower() for x in sym]

    i = 12
    roman_numeral = ''
    while number:
        div = number // num[i]
        number %= num[i]
  
        while div:
            roman_numeral = roman_numeral + sym[i]
            div -= 1
        i -= 1
    
    return roman_numeral

In [173]:
import os
os.getcwd()

'D:\\Content\\GoogleDrive\\Github\\mutual_fund_analysis\\notebooks'

In [174]:
df_sep = pd.read_excel('https://portal.amfiindia.com/spages/amsep2022repo.xls')

In [190]:
df_sep.columns = ['Sr', 'Category', 'NumSchemes', 'NumFolios', 'InFlow', 'OutFlow', 'NetInFlow', 'NetAUM', 'AverageAUM', 'NumSegPortfolios', 'AUMSegPortfolios']
df_s = df_sep.iloc[2:, 1:]

In [191]:
df_category = pd.read_csv('../mf_categories.csv')

In [194]:
df_x = df_s.merge(df_category, on='Category')
df_x['Month'] = 9
df_x['Year'] = 2022

In [195]:
df_x

,Category,NumSchemes,NumFolios,InFlow,OutFlow,NetInFlow,NetAUM,AverageAUM,NumSegPortfolios,AUMSegPortfolios,Class,Structure,Month,Year
0,Overnight Fund,32,611525,552502.991719,519374.658639,33128.33308,141319.784048,133564.031977,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
1,Liquid Fund,37,1753272,299102.844183,359073.141476,-59970.297293,352213.142253,419855.510365,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
2,Ultra Short Duration Fund,26,621847,13749.007249,22202.813311,-8453.806062,89480.736951,96696.217402,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
3,Low Duration Fund,22,986513,7331.801119,14991.647854,-7659.846736,95673.47909,101883.125742,1,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
4,Money Market Fund,22,420264,21219.657995,32451.719748,-11232.061754,107379.297939,115590.114297,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
5,Short Duration Fund,26,530585,3508.970477,4539.825665,-1030.855188,97272.496918,99473.387959,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
6,Medium Duration Fund,15,258984,266.209468,935.289637,-669.080169,28203.83209,28727.488527,3,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
7,Medium to Long Duration Fund,12,108680,43.509468,133.088444,-89.578976,8926.795229,9029.591409,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
8,Long Duration Fund,3,26259,154.4908,43.7344,110.7564,2828.4348,2772.2483,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022
9,Dynamic Bond Fund,24,218018,355.048652,1108.433581,-753.384929,22083.904658,22614.039139,0,0,Income/Debt Oriented Schemes,Open ended Schemes,9,2022


In [284]:
df_filenames = pd.read_excel('../file_names.xlsx')
df_category = pd.read_csv('../mf_categories.csv')

In [286]:
import datetime
import plotly_express as px
from pandas.tseries.offsets import MonthEnd

In [288]:
list_monthly = []
for index, row in df_filenames.iterrows():
    if index < 12:
        continue        
    url = 'https://portal.amfiindia.com/spages/' + row['File']
    df_month_data = pd.read_excel(url)
    df_month_data.columns = ['Sr', 'Category', 'NumSchemes', 'NumFolios', 'InFlow', 
                             'OutFlow', 'NetInFlow', 'NetAUM', 'AverageAUM', 
                             'NumSegPortfolios', 'AUMSegPortfolios']
    df_data = df_month_data.iloc[2:, 1:]
    df_data = df_data.merge(df_category, on='Category')
    df_data.loc[df_data['NetAUM'] == df_data[df_data['Category'] == 'ELSS'].groupby(['Class'])['NetAUM'].max().values[0], 'Structure'] = 'Open Ended Schemes'
    df_data.loc[df_data['NetAUM'] == df_data[df_data['Category'] == 'ELSS'].groupby(['Class'])['NetAUM'].min().values[0], 'Structure'] = 'Close Ended Schemes'
    df_data = df_data.drop_duplicates()
    df_data['MonthNumber'] = row['MonthNumber']
    df_data['Year'] = row['Year']
    df_data['MonthName'] = datetime.date(1900, row['MonthNumber'], 1).strftime('%B')
    list_monthly.append(df_data)

In [296]:
df_mf_aum = pd.concat(list_monthly)

In [298]:
df_mf_aum['mn'] = df_mf_aum['MonthNumber'].astype(str)
df_mf_aum['mn'] = df_mf_aum['mn'].map(lambda x: x.zfill(2))
df_mf_aum['ymn'] = df_mf_aum['Year'].astype(str) + df_mf_aum['mn']
df_mf_aum['Date'] = pd.to_datetime(df_mf_aum['ymn'], format="%Y%m") + MonthEnd(0)

In [326]:
sel_structure = 'Open Ended Schemes'
sel_class = 'Growth/Equity Oriented Schemes'
sel_measure = 'NetInFlow'
df_sel = df_mf_aum[(df_mf_aum['Class'] == sel_class) & (df_mf_aum['Structure'] == sel_structure)][['Category', sel_measure, 'Date']]
df_sel_long = pd.melt(df_sel, id_vars=['Category', 'Date'], value_vars=[sel_measure])
px.line(df_sel_long, x='Date', y='value', color='Category', labels={'value':sel_measure+'(Cr)'})

In [312]:
df_inflows = pd.DataFrame(df_mf_aum.groupby(['Structure', 'Class', 'Date'])['NetInFlow'].aggregate('sum')).reset_index()

In [313]:
df_inflows.head()

,Structure,Class,Date,NetInFlow
0,Close Ended Schemes,Growth/Equity Oriented Schemes,2021-04-30,-1654.248521
1,Close Ended Schemes,Growth/Equity Oriented Schemes,2021-05-31,-847.500755
2,Close Ended Schemes,Growth/Equity Oriented Schemes,2021-06-30,-1379.420563
3,Close Ended Schemes,Growth/Equity Oriented Schemes,2021-07-31,-1840.751883
4,Close Ended Schemes,Growth/Equity Oriented Schemes,2021-08-31,-609.888239


In [327]:
sel_structure = 'Open Ended Schemes'
sel_class = 'Growth/Equity Oriented Schemes'
#sel_class = 'Income/Debt Oriented Schemes'
sel_measure = 'NetInFlow'
df_selected = df_inflows[(df_inflows['Class'] == sel_class) & (df_inflows['Structure'] == sel_structure)]
px.line(df_selected, x='Date', y=sel_measure, color='Class', labels={sel_measure:sel_measure + '(Cr)'})